# Conversational Search

In this notebook you will implement the following steps:

- **Answer selection + evaluation**: Implement a *search-based* conversation framework evaluation framework to evaluate conversation topics made up of conversation turns.
- **Answer ranking**: Implement a *re-ranking method* to sort the initial search results. Evaluate the re-ranked results.
- **Conversation context**: Implement a conversational context modeling method to keep track of the conversation state. 

Submission dates:
- **20 October**: retrieval + evaluation
- **20 November**: pass
age re-ranking
- **20 December**: conversation state tracking

## Test bed and conversation topics
The TREC CAST corpus (http://www.treccast.ai/) for Conversational Search is indexed in this cluster and available to be searched behind an ElasticSearch API.

The queries and the relevance judgments are available through class `ConvSearchEvaluation`:

# Google Colab Setup

The following steps are already implemented in the cell bellow. You need to download the starting project folder, upload it, adjust the paths, and finally run the notebook.


1.   Download the shared project folder as a zip;
2.   Unzip and re-upload to a folder of your own GDrive;
3.   Mount your GDrive on the Colab working environment;

Note: You will be asked to complete a Google Authorization procedure by following a link and pasting a code on the notebook.

4.   Copy the contents from the folder you uploaded to the Colab working dir;
5.   Add sys path locations to run aux Python scripts;
6.   Install dependencies.

After going though all these steps you should be able to run all the cells in the notebook.

In [ ]:
# Colab Setup
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

# After downloading the shared starting point folder as a Zip
# Unzip it and re-upload it to a location on your GDrive

# This command copies the contents from the folder you uploaded to GDrive, to the colab working dir
!cp -r /content/drive/My\ Drive/**YOUR_PATH_TO_THE_PROJECT_FOLDER_ON_GDRIVE** /content

# Add working dir to the sys path, so that we can find the aux python files when running the Notebook
import sys
if not '/content/ProjectoRI2020' in sys.path:
  sys.path += ['/content/ProjectoRI2020']

# Finally install required dependencies to run the notebook
!pip install elasticsearch
!pip install bert-serving-client

In [ ]:
import TRECCASTeval as trec
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

import ElasticSearchSimpleAPI as es
import numpy as np

import pprint as pprint

test_bed = trec.ConvSearchEvaluation()

print()
print("========================================== Training conversations =====")
topics = {}
for topic in test_bed.train_topics:
    conv_id = topic['number']

    if conv_id not in (1, 2, 4, 7, 15, 17,18,22,23,24,25,27,30):
        continue

    print()
    print(conv_id, "  ", topic['title'])

    for turn in topic['turn']:
        turn_id = turn['number']
        utterance = turn['raw_utterance']
        topic_turn_id = '%d_%d'% (conv_id, turn_id)
        
        print(topic_turn_id, utterance)
        topics[topic_turn_id] = utterance

print()
print("========================================== Test conversations =====")
for topic in test_bed.test_topics:
    conv_id = topic['number']

    if conv_id not in (31, 32, 33, 34, 37, 40, 49, 50, 54, 56, 58, 59, 61, 67, 68, 69, 75, 77, 78, 79):
        continue

    print()
    print(conv_id, "  ", topic['title'])

    for turn in topic['turn']:
        turn_id = turn['number']
        utterance = turn['raw_utterance']
        topic_turn_id = '%d_%d'% (conv_id, turn_id)
        
        print(topic_turn_id, utterance)
        topics[topic_turn_id] = utterance


Search example:

In [ ]:
elastic = es.ESSimpleAPI()
results = elastic.search_body(topics['33_1'], numDocs = 10)
print(results)

## Retrieval with the training conversations
The ElasticSearchSimpleAPI notebook illustrates how to use ElasticSearch. Use this API to retrieve the top 100 ranked passages for each conversation turn. 

To evaluate the results you should use the provided `ConvSearchEvaluation` class. Examine and discuss the recall metric results. In terms of metrics, discuss what should be your goals for each step of the project.

In [ ]:
import TRECCASTeval as trec
import numpy as np

import ElasticSearchSimpleAPI as es
import numpy as np

import pprint as pprint

elastic = es.ESSimpleAPI()
test_bed = trec.ConvSearchEvaluation()

_recall = 0
_p10 = 0
_ndcg5 = 0

_nturns = 0

for topic in test_bed.train_topics:
    
    conv_id = topic['number']
    if conv_id not in (1, 2, 4, 7, 15, 17,18,22,23,24,25,27,30):
        continue
        
    for turn in topic['turn']:
        turn_id = turn['number']
        utterance = turn['raw_utterance']
        topic_turn_id = '%d_%d'% (conv_id, turn_id)
        
        aux = test_bed.relevance_judgments.loc[test_bed.relevance_judgments['topic_turn_id'] == (topic_turn_id)]
        num_rel = aux.loc[aux['rel'] != 0]['docid'].count()
        
        if num_rel == 0:
            continue

        result = elastic.search_body(query=utterance, numDocs = 100)

        if np.size(result) == 0 or num_rel == 0:
            print(topic_turn_id, utterance, num_rel, "NO RESULTS")
            continue
        else:
            print(topic_turn_id, utterance, num_rel)

        [p10, recall, ap, ndcg5] = test_bed.eval(result[['_id','_score']], topic_turn_id)

#        print('P10=', p10, '  Recall=', recall, '  NDCG=',ndcg)
        _recall = _recall + recall
        _p10 = _p10 + p10
        _ndcg5 = _ndcg5 + ndcg5
        
        _nturns = _nturns + 1


_p10 = _p10/_nturns
_recall = _recall/_nturns
_ndcg5 = _ndcg5/_nturns

print()
print('P10=', _p10, '  Recall=', _recall, '  NDCG@5', _ndcg5)


## Retrieval with the test conversations

In [ ]:
import TRECCASTeval as trec
import numpy as np

from bert_serving.client import BertClient
from sklearn.metrics.pairwise import cosine_similarity

import ElasticSearchSimpleAPI as es
import numpy as np

import pprint as pprint

elastic = es.ESSimpleAPI()
test_bed = trec.ConvSearchEvaluation()

_recall = 0
_p10 = 0
_ndcg5 = 0
_nturns = 0
for topic in test_bed.test_topics:
    
    conv_id = topic['number']
    if conv_id not in (31, 32, 33, 34, 37, 40, 49, 50, 54, 56, 58, 59, 61, 67, 68, 69, 75, 77, 78, 79):
        continue

    for turn in topic['turn']:
        turn_id = turn['number']
        utterance = turn['raw_utterance']
        topic_turn_id = '%d_%d'% (conv_id, turn_id)
        
        aux = test_bed.test_relevance_judgments.loc[test_bed.test_relevance_judgments['topic_turn_id'] == (topic_turn_id)]
        num_rel = aux.loc[aux['rel'] != 0]['docid'].count()
        
        if num_rel == 0:
            continue

        result = elastic.search_body(query=utterance, numDocs = 100)

        if np.size(result) == 0 or num_rel == 0:
            print(topic_turn_id, utterance, num_rel, "NO RESULTS")
            continue
        else:
            print(topic_turn_id, utterance, num_rel)

        [p10, recall, ap, ndcg5] = test_bed.eval(result[['_id','_score']], topic_turn_id)

#        print('P10=', p10, '  Recall=', recall, '  NDCG=',ndcg)
        _recall = _recall + recall
        _p10 = _p10 + p10
        _ndcg5 = _ndcg5 + ndcg5
        
        _nturns = _nturns + 1

_p10 = _p10/_nturns
_recall = _recall/_nturns
_ndcg5 = _ndcg5/_nturns

print()
print('P10=', _p10, '  Recall=', _recall, '  NDCG@5', _ndcg5)


## Passage re-Ranking
The Passage Ranking notebook example illustrates how to use the BERT service to compute the similarity between sentences. Using the BERT service, improve a passage ranking method to rerank the initial retrieval step.

To evaluate the results you should use the provided `ConvSearchEvaluation` class.


## Conversation Context Modeling

Conversation State Tracking example ilustrates how to use the 

To evaluate the results you should use the provided `ConvSearchEvaluation` class.
